In [4]:
from tqdm import tqdm
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings("ignore")
PATH = 'C:/Program Files/chromedriver.exe'

## Some functions

In [5]:
def open_url(url, pageLoadStrategy='eager'):
    caps = DesiredCapabilities().CHROME ; caps["pageLoadStrategy"] = pageLoadStrategy # "eager", "normal", "none"
    chrome_options = Options()  ; chrome_options.add_argument("window-size=700,800")

    driver = webdriver.Chrome(executable_path=PATH, desired_capabilities=caps, chrome_options=chrome_options)
    driver.minimize_window()
    driver.get(url)
    try:
        expand = driver.find_element(by=By.CLASS_NAME, value='toggle_expand')
        expand.click()
    except: pass
    return driver

def get_element_by_xpath(driver, xpath):
    return driver.find_element(by=By.XPATH, value=xpath)

def all_info(driver):
    """Summary (info)"""
    info = get_element_by_xpath(driver, '//*[@id="main"]/div/div[1]/div[1]').text.lower().split('\n')
                                        
    """Metascore"""
    try:
        meta_positive = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[1]/div[2]/div/div/ol/li[1]').text.text.split('\n')[1]
        meta_mixed = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[1]/div[2]/div/div/ol/li[2]').text.text.split('\n')[1]
        meta_negative = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[1]/div[2]/div/div/ol/li[3]').text.text.split('\n')[1]
    except:
        meta_positive = '0'
        meta_mixed = '0'
        meta_negative = '0'
    
    """Userscore"""
    try:
        user_positive = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[2]/div[3]/div/div/ol/li[1]/div/span[2]/a/span/span[1]').text
        user_mixed = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[2]/div[3]/div/div/ol/li[2]/div/span[2]/a/span/span[1]').text
        user_negative = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[1]/div[1]/div/div[2]/div[3]/div/div/ol/li[3]/div/span[2]/a/span/span[1]').text
    except:
        user_positive = '0'
        user_mixed = '0'
        user_negative = '0'
    """Awards"""
    awards = get_element_by_xpath(driver, '//*[@id="main"]/div/div[3]/div[2]/div[1]/div[2]').text

    return info, meta_positive, meta_mixed, meta_negative, user_positive, user_mixed, user_negative, awards

def del_item(arr, items):
    for item in items:
        try: arr.remove(item.lower())
        except: pass
    return arr

In [6]:
url1 = 'https://www.metacritic.com/game/xbox-series-x/f1-2021'
url2 = 'https://www.metacritic.com/game/xbox-series-x/immortality'
url3 = 'https://www.metacritic.com/game/xbox-series-x/elden-ring'
url4 = 'https://www.metacritic.com/game/xbox-series-x/martha-is-dead'
url5 = 'https://www.metacritic.com/game/xbox-series-x/halo-infinite'
url6 = 'https://www.metacritic.com/game/xbox-series-x/hades'
url7 = 'https://www.metacritic.com/game/xbox-series-x/forza-horizon-5'

urls = [url1, url2, url3, url4, url5, url6, url7,
        'https://www.metacritic.com/game/xbox-series-x/rogue-legacy-2',
        'https://www.metacritic.com/game/xbox-series-x/atari-50-the-anniversary-celebration',
        'https://www.metacritic.com/game/xbox-series-x/life-is-strange-true-colors',
        'https://www.metacritic.com/game/xbox-series-x/far-cry-6',
        'https://www.metacritic.com/game/xbox-series-x/spellforce-iii-reforced',
        'https://www.metacritic.com/game/xbox-series-x/dying-light-2-stay-human',
        'https://www.metacritic.com/game/xbox-series-x/monster-energy-supercross---the-official-videogame-5',
        'https://www.metacritic.com/game/xbox-series-x/the-dark-pictures-anthology-house-of-ashes',
        'https://www.metacritic.com/game/xbox-series-x/vampire-the-masquerade---swansong',
        'https://www.metacritic.com/game/xbox-series-x/werewolf-the-apocalypse---earthblood',
        'https://www.metacritic.com/game/xbox-series-x/crossfirex']

items_to_remove = ['summary','critic reviews','user reviews','details & credits',
                   "what's this?", 'trailers & videos', '… expand', 'buy now', 'buy on', 'your score', '0', 
                   'cheats: on gamefaqs', 'more details and credits »', 'trailer', ]

In [7]:
os.makedirs('InfoText', exist_ok=True)

for url in tqdm(urls[:]):
    driver = open_url(url, pageLoadStrategy='eager')
    info, meta_positive, meta_mixed, meta_negative, user_positive, user_mixed, user_negative, awards = all_info(driver)
    info = list(filter(('').__ne__, info))
    info = del_item(info, items_to_remove)
    info = [item for item in info if ('also on' not in item and 'available on:' not in item)] # delete items including "also on"
    driver.close()
    f = open(f"InfoText/{url.split('/')[-1]}.txt", 'w')
    f.write('\n'.join(info))
    f.close()

100%|██████████| 18/18 [03:00<00:00, 10.01s/it]


## Test one by one

In [83]:
driver = open_url('https://www.metacritic.com/game/xbox-series-x/life-is-strange-true-colors')
el = get_element_by_xpath(driver, '//*[@id="main"]/div/div[1]/div[1]').text.lower().split('\n')

In [89]:
info = el
info = list(filter(('').__ne__, info))
info = del_item(info, items_to_remove)
info = [item for item in info if ('also on' not in item and 'available on:' not in item)]
info

['life is strange: true colors xbox series x',
 'square enix',
 'release date: sep 10, 2021',
 'metascore',
 '84',
 'generally favorable reviews',
 'based on 17 critic reviews',
 'user score',
 '6.3',
 'mixed or average reviews',
 'based on 120 ratings',
 'summary: a bold new era of the award-winning life is strange begins, with an all-new playable lead character and a thrilling mystery to solve!',
 "alex chen has long suppressed her 'curse': the supernatural ability to experience, absorb and manipulate the strong emotions of others, which she sees as",
 'developer: deck nine games',
 'genre(s): action adventure, general',
 '# of players: no online multiplayer',
 'rating: m']

In [77]:
el.text.lower().split('\n')

['life is strange: true colors xbox series x',
 'square enix',
 'release date: sep 10, 2021',
 'also on: pc, playstation 4, playstation 5, stadia, switch, xbox one',
 'summary',
 'critic reviews',
 'user reviews',
 'details & credits',
 'trailers & videos',
 'metascore',
 '84',
 'generally favorable reviews',
 'based on 17 critic reviews',
 "what's this?",
 'user score',
 '6.3',
 'mixed or average reviews',
 'based on 120 ratings',
 'your score',
 '0',
 'summary: a bold new era of the award-winning life is strange begins, with an all-new playable lead character and a thrilling mystery to solve!',
 '',
 "alex chen has long suppressed her 'curse': the supernatural ability to experience, absorb and manipulate the strong emotions of others, which she sees as",
 '',
 '',
 '',
 '… expand',
 'buy now',
 'buy on',
 'developer: deck nine games',
 'genre(s): action adventure, general',
 '# of players: no online multiplayer',
 'cheats: on gamefaqs',
 'available on: stadia',
 'rating: m',
 'more d